In [23]:
from datetime import datetime
import boto3
import botocore
from botocore.client import Config
import warnings
import pyart
import os
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
s3 = boto3.resource('s3', config=Config(signature_version=botocore.UNSIGNED, \
                                    user_agent_extra='Resource'))
bucket = s3.Bucket('s3-radaresideam')

In [3]:
!aws s3 ls s3://s3-radaresideam/l2_data/2021/10/03/ --no-sign-request

                           PRE Barrancabermeja/
                           PRE Guaviare/
                           PRE Munchique/


In [4]:
d = datetime(2021, 10, 3, 12)
radar_station = 'Guaviare'

In [6]:
prefix = f'l2_data/{d:%Y}/{d:%m}/{d:%d}/{radar_station}/{radar_station[:3].upper()}{d:%y%m%d%H}'
prefix

'l2_data/2021/10/03/Guaviare/GUA21100312'

In [26]:
objects = []

for obj in bucket.objects.filter(Prefix=prefix):
    print(obj.key)
    objects.append(obj)
    

l2_data/2021/10/03/Guaviare/GUA211003120004.RAWTNBC
l2_data/2021/10/03/Guaviare/GUA211003120124.RAWTNBF
l2_data/2021/10/03/Guaviare/GUA211003120316.RAWTNBH
l2_data/2021/10/03/Guaviare/GUA211003120504.RAWTNBR
l2_data/2021/10/03/Guaviare/GUA211003120624.RAWTNBV
l2_data/2021/10/03/Guaviare/GUA211003120816.RAWTNBX
l2_data/2021/10/03/Guaviare/GUA211003121004.RAWTNC4
l2_data/2021/10/03/Guaviare/GUA211003121124.RAWTNC7
l2_data/2021/10/03/Guaviare/GUA211003121315.RAWTNC9
l2_data/2021/10/03/Guaviare/GUA211003121504.RAWTNCF
l2_data/2021/10/03/Guaviare/GUA211003121623.RAWTNCJ
l2_data/2021/10/03/Guaviare/GUA211003121816.RAWTNCL
l2_data/2021/10/03/Guaviare/GUA211003122004.RAWTNCU
l2_data/2021/10/03/Guaviare/GUA211003122123.RAWTNCY
l2_data/2021/10/03/Guaviare/GUA211003122315.RAWTND0
l2_data/2021/10/03/Guaviare/GUA211003122504.RAWTND7
l2_data/2021/10/03/Guaviare/GUA211003122622.RAWTNDA
l2_data/2021/10/03/Guaviare/GUA211003122814.RAWTNDC
l2_data/2021/10/03/Guaviare/GUA211003123004.RAWTNDG
l2_data/2021

In [21]:
objects[0]

s3.ObjectSummary(bucket_name='s3-radaresideam', key='l2_data/2021/10/03/Guaviare/GUA211003120004.RAWTNBC')

In [25]:
for obj in objects:
    paht, filename = os.path.split(obj.key)
    bucket.download_file(obj.key, filename)


In [29]:
import glob
glob.glob(f'{os.getcwd()}/GUA*')


['C:\\Users\\ladin\\Downloads\\GUA211003120004.RAWTNBC',
 'C:\\Users\\ladin\\Downloads\\GUA211003120124.RAWTNBF',
 'C:\\Users\\ladin\\Downloads\\GUA211003120316.RAWTNBH',
 'C:\\Users\\ladin\\Downloads\\GUA211003120504.RAWTNBR',
 'C:\\Users\\ladin\\Downloads\\GUA211003120624.RAWTNBV',
 'C:\\Users\\ladin\\Downloads\\GUA211003120816.RAWTNBX',
 'C:\\Users\\ladin\\Downloads\\GUA211003121004.RAWTNC4',
 'C:\\Users\\ladin\\Downloads\\GUA211003121124.RAWTNC7',
 'C:\\Users\\ladin\\Downloads\\GUA211003121315.RAWTNC9',
 'C:\\Users\\ladin\\Downloads\\GUA211003121504.RAWTNCF',
 'C:\\Users\\ladin\\Downloads\\GUA211003121623.RAWTNCJ',
 'C:\\Users\\ladin\\Downloads\\GUA211003121816.RAWTNCL',
 'C:\\Users\\ladin\\Downloads\\GUA211003122004.RAWTNCU',
 'C:\\Users\\ladin\\Downloads\\GUA211003122123.RAWTNCY',
 'C:\\Users\\ladin\\Downloads\\GUA211003122315.RAWTND0',
 'C:\\Users\\ladin\\Downloads\\GUA211003122504.RAWTND7',
 'C:\\Users\\ladin\\Downloads\\GUA211003122622.RAWTNDA',
 'C:\\Users\\ladin\\Downloads\\